# Final Assignment 18145399

# Introduction
## Aims and Objectives
In this assessment, I will code a game in Python 3 kernel with mechanics similar to “Angry Birds”, in which users can enter the parameters of initial velocity, which are its angle $\alpha$ with the horizontal ground and its magnitude $v_0$, to launch a projectile at a target to try to topple it. In order to do this, I am going to write the codes for:

• showing the path of the projectile;

• calculating the momentum of the object at the point of collision with the target, the impulse transferred to the target as well as the torque applying to the target due to the collision, and whether or not this is greater than the restoring torque to make target be toppled.

• showing the animation of the whole process.

## Theory
In this assignment, we would use following equation to calculate the displacement, velocity, momentum of birds, applied force and torque to target as weel as the magnitude of restoring torque.

For path calculating, we get:
$$  
r_x = r_{x0}+v_0cos\theta dt \\
r_y = r_{y0}+v_0sin\theta dt - \frac{gt^2}{2} \\
\vec{position} = (r_x, r_y, 0)
$$

For momentum, we obtain:
$$ \vec{p} = m\frac{d\vec{r}}{dt}= m\vec{v} $$
$$ v_x = v_0cos\theta  $$
$$ v_y = v_0sin\theta - gdt $$
$$ p_x = mv_x = mv_0cos\theta  $$
$$ p_y = mv_y = mv_0sin\theta - mgdt $$

We assume that the collision between the target and bird is complete elatstic, and the total momentum of bird would transfer to the target during the finite contact time $\delta t_{collision}$,

The force can be calculated from the impulse:
$$ \vec{p}= m\vec{v}$$
$$\delta{\vec{p}} = \vec{I}$$
$$  \vec{F_{applied}​} =  \frac{\vec{I}}{\delta t_{collision}} =  \frac{ m\vec{v}}{\delta t_{collision}}$$

,which are:
$$F_x =  \frac{mv_o\cos\theta}{\delta t_{collision}}$$
$$ F_y =  \frac{m(v_o\sin\theta -gt)}{\delta t_{collision}}$$
$$\vec{F_{applied}}=(F_x, F_y, 0)$$
During the collsion(before target toppled with an angle), there would be a applied torque to make the target fall against restoring torque relevant to the gravity:
$$|\vec{t_{restoring}}|= -Mg\frac{w}{2}$$
$$ \vec{d_a} = (w, -h ,0)$$
$$\vec{t_{applied}} = \vec{F_{applied}}\times \vec{d_a}$$
$$|\vec{t_{applied}}| = |\vec{F_{applied}}\times \vec{d_a}|= |(F_x, F_y, 0)\times (w, -h ,0)|$$

When $|\vec{t_{restoring}}|$ smaller than $|\vec{t_{applied}}|$, the target toppled.

In [2]:
# import Vpython function
import numpy as np
from vpython import  sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, arrow, radians

<IPython.core.display.Javascript object>

In [ ]:
def initial_frame():
    scene = canvas(width=640, height=480, center=vector(8,5,0),range=8)
    ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)
    label(pos=vector(8,-2,0), text='Welcome to My Game', font='sans')
    bird = sphere(pos = vector(0,random(),0),radius= 0.3,color=color.blue,make_trail=False) 
    bird.trail_color = color.white
    momentum_vector = arrow(pos = bird.pos,axis= (vector(vx,vy,0)), shaftwidth=0.1, 
                    headwidth = 0.15, headlength = 0.15,color=color.blue)
def generate_bird(rx,ry,trial):
    bird = sphere(pos = vector(rx,ry,0),radius= 0.3,color=color.blue,make_trail=trail)
    bird.trial_color=color.white
    momentum_vector = arrow(pos = bird.pos,axis= (vector(vx,vy,0)), shaftwidth=0.1, 
                headwidth = 0.15, headlength = 0.15,color=color.blue)
def generate_target():
    target = 

In [7]:
# ------------------------------------set up for constants---------------------------------------------------------
# Part of codes in following are copied from the Session 9 script and work done by me in Session 9 and Session 8
# the followings are in units where one screen unit is equal to one metre 
# Define some physical quantities and constants
M = 100       # mass of target (in units kg)
m = 0.1       # mass of bird (in units kg)
g = 9.81      # gravitational acceleration (in units m/s^2)
tc = 0.01 # contact time of collision
w = 0.5 # width of target 
t = 0 # time period before bird fall on the ground
dt = 0.0001 # timestep set for path drawing

#---------------------------------------set up for canvas, bird and target------------------------------------------------------------

def canvas_set():
    '''function that creates basic set which is need to be
     repeated when hitting or toppling fails and there is no needs for inputs.'''
    # create a canvas for the picture 
    scene = canvas(width=640, height=480, center=vector(8,5,0),range=8)
    # create a “ground” near the bottom of the canvas
    ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)
    # label for the canvas
    label(pos=vector(8,-2,0), text='Welcome to My Game', font='sans')
    # random x-position of target
    xtarget = 5+random()*10    
    # set target 
    target = box(pos=vector(xtarget,1,0), length=0.5, height=2, width=0.5) 
    # set platform for launching
    y = random() # random y-position of platform
    platform = curve(pos=[(-0.4,y,0),(0.4,y,0)],color=color.green) # set platform
    # set other quantities 
    rx = 0 # original x-position of bird
    ry = y+0.15 # original y-position of bird
    h = ry # height of where collision happen 
    d = vector(w,-h,0) # displacement vector of force applied
    # set the bird with its moving trail
    bird = sphere(pos = vector(rx,ry,0),radius= 0.3,color=color.blue,make_trail=False) 
    # change trail colour
    bird.trail_color = color.white
    # set arrow showing the size and direction of momentum of bird
    momentum_vector = arrow(pos = bird.pos,axis= (vector(vx,vy,0)), shaftwidth=0.1, 
                    headwidth = 0.15, headlength = 0.15,color=color.blue)

    #--------------------------------------------inputs of parameters for the launching-------------------------------------------

    # input parameter v0 and theta for initial velocity
v0 = float(input("Please input the initial speed in m/s: "))
thetad = float(input("Please input the initial angle in degree: ")) 
# original angle (in radians) of input velocity with the horizontal ground 
e = np.radians(thetad) 

# define the velocity, force and torque based on the inputs
vx = v0*np.cos(e) # x-direction magnitude of velocity of bird
vy = v0*np.sin(e)-g*t# y-direction magnitude of velocity of bird 
fx = (m*vx)/tc  # x-direction magnitude of applied force vector 
fy = (m*vy)/tc # y-direction magnitude of applied force vector
F = vector(fx,fy,0)   # applied force vector   
ta = F.cross(d) # torque applied (applied force corss product displacement vector)
mta = mag(ta)  #magnitude of torque applied
mtr = M*g*w/2  #magnitude of torque restoring






Please input the initial speed in m/s: 10
Please input the initial angle in degree: 30


NameError: name 'd' is not defined

In [10]:
def set_canvas(rx, ry, d):
    '''function that creates basic set which is need to be
     repeated when hitting or toppling fails and there is no needs for inputs.'''
    # create a canvas for the picture 
    scene = canvas(width=640, height=480, center=vector(8,5,0),range=8)
    # create a “ground” near the bottom of the canvas
    ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)
    # label for the canvas
    label(pos=vector(8,-2,0), text='Welcome to My Game', font='sans')
    # random x-position of target
    xtarget = 5+random()*10    
    # set target 
    target = box(pos=vector(xtarget,1,0), length=0.5, height=2, width=0.5) 
    # set platform for launching
    y = random() # random y-position of platform
    platform = curve(pos=[(-0.4,y,0),(0.4,y,0)],color=color.green) # set platform
    # set other quantities 
    ry = y+0.15 # original y-position of bird
    h = ry # height of where collision happen 
    d = vector(w,-h,0) # displacement vector of force applied
    # set the bird with its moving trail
    bird = sphere(pos = vector(rx,ry,0),radius= 0.3,color=color.blue,make_trail=False) 
    # change trail colour
    bird.trail_color = color.white
    # set arrow showing the size and direction of momentum of bird
    momentum_vector = arrow(pos = bird.pos,axis= (vector(vx,vy,0)), shaftwidth=0.1, 
                    headwidth = 0.15, headlength = 0.15,color=color.blue)

In [13]:
set_canvas(1,1,vector(0,0,0))

<IPython.core.display.Javascript object>

In [6]:
#-----------------------------------------begin the launching-----------------------------------------------------
# for the motion does not exceed the canvas
ry = 0
rx=0
while ry>=0 and rx<=16: 
    canvas_set()
    bird.make_trail = True
    rate(500) # control the rate of motion
    t = t + dt #acculmulating the time before bird fall on the ground
    rx = rx + vx*t # accumulate x value
    ry = ry + vy*t-(g*t**2)/2  # accumulate x value
    vx = vx # vx remains the same
    vy = vy - g*t  # accumulate vy value
    bird.pos = vector(rx,ry,0) # change birds position
    momentum_vector.pos = vector(rx,ry,0)  # change arrow position
    momentum_vector.axis = vector(vx,vy,0)# change arrow direction 

    #define if bird hits the target
    if xtarget-0.25 <= rx and rx <= 0.25+xtarget and  ry<=2: 
        # let bird land at that x-position with y-position eqaul to 0
        rx = rx 
        ry = 0
        bird.pos = vector(rx,ry,0)
        # let arrow disappear
        vx = 0 
        vy = 0
        momentum_vector.pos = vector(rx,ry,0)
        momentum_vector.axis = vector(vx,vy,0)
        #('Congratulation! You hit the target!')

        # after hitting, we define whether bird topples the target by comparing the torques
        if mta >= mtr:
            target.rotate(angle=-np.pi/2,axis=vector(0,0,1), origin=vector(xtarget,0.25,0))
            label(pos=vector(8,8,0), text='Congratulation! You Win!', font='sans')
        else:
            choice = float(input("1Would you like to try again?(Please enter Y/F)"))
            if choice == Y:
                set() #reset the canvas
                bird.make_trail = False #clear the trail

            else:
                label(pos=vector(8,8,0), text='Thanks for playing.', font='sans') #quit the game
    else:
        rx = rx + vx*dt # accumulate x value
        ry = ry + vy*dt-(g*dt**2)/2  # accumulate x value
        vy = vy - g*dt  # accumulate vy value
        bird.pos = vector(rx,ry,0) # change birds position
        momentum_vector.pos = vector(rx,ry,0)
        momentum_vector.axis = vector(vx,vy,0)# change arrow direction
        if ry == 0:
            choice = float(input("2Would you like to try again?(Please enter Y/F)"))
            if choice == Y:
                set() #reset the canvas
                bird.make_trail = False #clear the trail
            else:
                print("Thanks for playing.")
    label(pos=vector(rx,ry,0), text='This is a box')

    #print("end of program")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

UnboundLocalError: local variable 'vx' referenced before assignment

In [69]:
scene = canvas(width=640, height=480, center=vector(8,5,0),range=8)
xtarget = 5+random()*10    
ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)
target = box(pos=vector(xtarget,1,0), length=0.5, height=2, width=0.5) 
bird2 = sphere(pos = target.pos,radius= 0.3,color=color.blue,make_trail=True) 

#a=
while a <= np.pi/2:
    print('1')
    rate(5)
    a = a +1
    target.rotate(angle=-a,axis=vector(0,0,1), origin=vector(xtarget,0.25,0))
#              target.pos)
              #)


<IPython.core.display.Javascript object>

# 

In [42]:
def iunu

float

0.6666666666666666

# Discussion

## Suggestion for improving the physics representation 
## Future implemention of above improvements